#  Look to see if there is a delay in achieving driving force equilibrum
Look at the amplitude of voltage clamp first pulses at the beginning of the experiemnt (before internal and external solutions have equalized) to see if they are increasing or decreasing.  

The following applies:
* pass qc
* sst to sst connections
* holding potential between -50 and -60 mV 

In [ ]:
# imports
import numpy as np
import matplotlib
font={'size' : 16}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
from multipatch_analysis.database import database as db
import multipatch_analysis.connection_strength as cs 
from multipatch_analysis.database.database import TableGroup

In [ ]:
#Query sst to sst connections
s = db.Session()
pre_cell = db.aliased(db.Cell)
post_cell = db.aliased(db.Cell)
pre_cre = 'sst'
post_cre = 'sst'

q = s.query(db.Pair, db.Experiment.acq_timestamp, pre_cell.ext_id, post_cell.ext_id, pre_cell.cre_type, post_cell.cre_type)\
    .join(pre_cell, db.Pair.pre_cell_id==pre_cell.id)\
    .join(post_cell, db.Pair.post_cell_id==post_cell.id)\
    .join(db.Experiment)

q_filter = [pre_cell.cre_type==pre_cre,\
           post_cell.cre_type==post_cre,\
           db.Pair.synapse==True]

for f in q_filter:
    q = q.filter(f)
    
sst = q.all()
print len(sst)

In [ ]:
def event_qc(events, criterion):
    mask = events[criterion] == True
    mask &= np.abs(events['baseline_current']) < 500e-12
    mask &= events['baseline_potential'] < -50e-3
    mask &= events['baseline_potential'] > -60e-3
    return mask 

def get_first_pulse_deconvolved_amp(uid, pre_cell_id, post_cell_id, get_data=False):
    session = db.Session()
    #note requerying the pair because the Session was going lazy
    expt = db.experiment_from_timestamp(uid, session=session)
    pair = expt.pairs[(pre_cell_id, post_cell_id)]
    
    # 1. Get a list of all presynaptic spike times and the amplitudes of postsynaptic responses    
    clamp_mode='vc'
    raw_events = cs.get_amps(session, pair, clamp_mode=clamp_mode, get_data=get_data)
    if ((pair.connection_strength.synapse_type == 'in') & (clamp_mode == 'ic')) or ((pair.connection_strength.synapse_type == 'ex') & (clamp_mode == 'vc')):
        amplitude_field = 'neg_dec_amp'
        criterion='in_qc_pass'
    elif ((pair.connection_strength.synapse_type == 'ex') & (clamp_mode == 'ic')) or ((pair.connection_strength.synapse_type == 'in') & (clamp_mode == 'vc')):  
        amplitude_field = 'pos_dec_amp'
        criterion='ex_qc_pass'
    else:
        raise Exception('huh?')
    pass_qc_mask = event_qc(raw_events, criterion)
    events = raw_events[pass_qc_mask]
    if pair.connection_strength is None:
        print ("\t\tSKIPPING: pair_id %s, uid %s, is not yielding pair.connection_strength" % (pair.id, uid))
        if get_data==True:
            return None, None, None, None
        else:
            return None, None, None
    print("TRYING: %0.3f, cell ids:%s %s" % (uid, pre_cell_id, post_cell_id))
    if len(events)==0:
        print ("\tno events: %0.3f, cell ids:%s %s" % (uid, pre_cell_id, post_cell_id))
        session.close()
        if get_data==True:
            return None, None, None, None
        else:
            return None, None, None
    
    rec_times = 1e-9 * (events['rec_start_time'].astype(float) - float(events['rec_start_time'][0]))
    spike_times = events['max_dvdt_time'] + rec_times

    raw_bg_events = cs.get_baseline_amps(session, pair, clamp_mode='ic')
    bg_qc_mask = event_qc(raw_bg_events, criterion)
    bg_events = raw_bg_events[bg_qc_mask]
    mean_bg_amp = bg_events[amplitude_field].mean()
    amplitudes = events[amplitude_field] - mean_bg_amp
    pulse_ids = events['id']

    first_pulse_mask = events['pulse_number'] == 1
    first_pulse_amps = amplitudes[first_pulse_mask]
    first_pulse_times = rec_times[first_pulse_mask]
    first_pulse_ids = pulse_ids[first_pulse_mask]
    if get_data==True:
        data = events['data'][first_pulse_mask]
        session.close()
        
        return first_pulse_times, first_pulse_amps, first_pulse_ids, data
    else:
        return first_pulse_times, first_pulse_amps, first_pulse_ids



In [ ]:
for p in sst:
    first_pulse_times, first_pulse_amps, first_pulse_ids, data = get_first_pulse_deconvolved_amp(p[1], p[2], p[3], get_data=True)
    if first_pulse_times is None:
        continue
    plt.figure(figsize=(12,5))
    small_t_mask=first_pulse_times<200
    fpt=first_pulse_times[small_t_mask]
    fpa=first_pulse_amps[small_t_mask]
    plt.plot(fpt, fpa, '.', ms=20)
#     for ii, (p_id,t,a) in enumerate(zip(first_pulse_ids, first_pulse_times, first_pulse_amps)):
#         plt.annotate(str(ii), xy=(t,a), textcoords='data')
    plt.yticks([])
    plt.ylabel('deconvolved amplitude')
    plt.xlabel('time (s)')
    plt.title('First Pulse Deconvolution Amplitude\n%.3f, pre %s, post %s' %(p[1], p[2], p[3]))
    
    plt.tight_layout()
    plt.show()
